In [84]:
# importing modules
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar

In [85]:
# Read the data and merge district level files
rheas_district= pd.read_csv('data/rheas/zambia_rheas_district_data_clean.csv')
preharvest_district=pd.read_csv('data/preharvest/preharvest_cleaned.csv')

# rename district columns
rheas_district.rename(columns={'district':'district_b'},inplace=True)
preharvest_district.rename(columns={'district':'district_b'},inplace=True)

# reas district shapefile
district_shp=gpd.read_file('data/shp/zmb_district_cleaned.shp')


In [86]:
# aggregate district level to one per district
rheas_district_grouped_grouped=rheas_district.groupby('district_b', as_index=False).agg({'yield':'mean'})
preharvest_district_grouped=preharvest_district.groupby('district_b', as_index=False).agg({'yield':'mean'})
# read rheas_preharvest_district_metrics
rheas_preharvest_district_metrics=pd.read_csv('data/metrics/preharvest_rheas_district_metrics.csv')

# rename preharvest yield column before merging
preharvest_district_grouped.rename(columns={'yield':'preharvest_yield', 'district_b':'district_p'}, inplace=True)

In [87]:
# extract rheas for the districts merging preharvest
preharvest_districts=list(preharvest_district_grouped.district_p.unique())
rheas_districts=list(rheas_district_grouped_grouped.district_b.unique())

list_rheas=[]

for district in rheas_districts:
    if district not in preharvest_districts:
        pass
    else:
        df_rheas=rheas_district_grouped_grouped.loc[rheas_district_grouped_grouped['district_b']==district]
        list_rheas.append(df_rheas)
rheas_district_grouped_grouped1=pd.concat(list_rheas)
rheas_district_grouped_grouped1

,district_b,yield
0,chadiza,4.002790
1,chama,4.021222
3,chavuma,4.022218
5,chibombo,4.009595
6,chiengi,4.017078
...,...,...
104,shangombo,4.021398
107,siavonga,4.015350
109,sinazongwe,3.997217
112,solwezi,4.004949


In [90]:
# merge rheas, preharvest and metrics
sort metrics data first
# rheas_district_grouped_grouped1
# preharvest_district_grouped
# rheas_preharvest_district_metrics
rheas_metrics=pd.concat([rheas_district_grouped_grouped1,
                                     preharvest_district_grouped,rheas_preharvest_district_metrics], axis=1)
rheas_metrics

,district_b,yield,district_p,preharvest_yield,district,correlation,rmse,ubrmse,bias
0,chadiza,4.002790,chadiza,2.510959,mazabuka,0.398194,1.336786,0.600286,1.194426
1,chama,4.021222,chama,2.639932,sesheke,-0.249971,3.030859,0.417523,3.001963
3,chavuma,4.022218,chibombo,2.541407,mufulira,-0.171500,0.752389,0.652796,0.374096
5,chibombo,4.009595,chililabombwe,2.855052,chiengi,-0.036587,1.423604,0.369104,1.374922
6,chiengi,4.017078,chilubi,2.778381,livingstone,0.287322,2.640326,0.782188,2.521806
...,...,...,...,...,...,...,...,...,...
56,NaN,NaN,mungwi,3.383704,lundazi,0.272211,1.443612,0.358537,1.398380
57,NaN,NaN,mwense,2.657257,chililabombwe,-0.302201,1.270573,0.557698,1.141634
59,NaN,NaN,nakonde,2.974662,nchelenge,-0.270999,1.389866,0.454236,1.313544
64,NaN,NaN,petauke,1.912877,solwezi,-0.598471,1.942293,0.466634,1.885405
